In [12]:
import pandas as pd 
import numpy as np 

import os 
from glob import glob 

from PIL import Image
import albumentations as A
from albumentations import ImageOnlyTransform
from augraphy import *

from tqdm import tqdm

In [13]:
# 잘못된 label 업데이트
train_df = pd.read_csv("data/train.csv")
train_df.loc[428, 'target'] = 7
train_df.loc[1095, 'target'] = 14
train_df.loc[862, 'target'] = 3
train_df.loc[192, 'target'] = 7
train_df.loc[1237, 'target'] = 14
train_df.loc[38, 'target'] = 10
train_df.loc[340, 'target'] = 10

train_df.to_csv("data/train.csv", index=False)

In [14]:
df = pd.read_csv('data/train.csv')
data_path = 'data/train/'

In [15]:
# 605 통원확인서
# 187 진료확인서
# 225 입-퇴원 확인서
# 1359 통원확인서
# 738 통원확인서
# rows_to_append =df.loc[[605, 187, 225, 1359, 738]]
# df = pd.concat([df, rows_to_append], ignore_index=True)

# df.shape

In [16]:

horizontal_flip = A.Compose([
    A.HorizontalFlip(p=1),
])

vertical_flip = A.Compose([
    A.VerticalFlip(p=1),
])

double_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

transpose = A.Compose([
    A.Transpose(p=1)
])

transpose_hflip = A.Compose([
    A.Transpose(p=1), 
    A.HorizontalFlip(p=1),
])

transpose_vflip = A.Compose([
    A.Transpose(p=1),
    A.VerticalFlip(p=1),
])

transpose_dflip = A.Compose([
    A.Transpose(p=1),  
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

pattern = PatternGenerator(
            imgx = 120,
            imgy = 120,
            n_rotation_range = (10, 15),
            alpha_range = (0.1, 0.2),
            color = (130, 130, 130),
        )

class PatternGeneratorTransform(ImageOnlyTransform):
    def __init__(self, pattern_generator, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.pattern_generator = pattern_generator

    def apply(self, img, **params):
        return self.pattern_generator(img)

noise_rotate = A.Compose([
    # A.GaussNoise(var_limit=(100, 1000), per_channel=False, p=1),
    # A.Blur(blur_limit=(2, 4), p=0.5),
    # A.ToGray(p=0.2),
    # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.4, border_mode=0, value=(255, 255, 255)),

    A.OneOf([A.GaussNoise(var_limit=(100, 900), per_channel=False, p=1), PatternGeneratorTransform(pattern, p=1)], p=0.8), 
    A.OneOf([A.MotionBlur(p=0.2), A.MedianBlur(blur_limit=3, p=0.2), A.Blur(blur_limit=(2, 4), p=1),], p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),    
    A.ToGray(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.5, border_mode=0, value=[255, 255, 255]),
])

In [17]:
base_aug_types = [
    ("hf_", horizontal_flip),
    ("vf_", vertical_flip),
    ("df_", double_flip),
    ("t_", transpose),
    ("thf_", transpose_hflip),
    ("tvf_", transpose_vflip),
    ("tdf_", transpose_dflip)
]

In [18]:
ids = []
targets = []
for index, ID, target in tqdm(df.itertuples(), total=df.shape[0], desc='Image augmentation'):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    
    for prefix, aug_function in base_aug_types:
        transformed_image = aug_function(image=image)['image']
        new_ID = prefix + ID
        ids.append(new_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, new_ID))
        
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_data = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_data])

Image augmentation:   1%|          | 12/1570 [00:00<00:28, 54.00it/s]

Image augmentation: 100%|██████████| 1570/1570 [00:22<00:00, 71.00it/s]


In [19]:
ids = []
targets = []
for index, ID, target in tqdm(df.itertuples(), total=df.shape[0], desc='Image augmentation'):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    for i in range(7):
        transformed_image = noise_rotate(image=image)['image']
        image_ID = f'nr{i}_' + ID 
        ids.append(image_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_data = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_data])

Image augmentation: 100%|██████████| 12560/12560 [36:21<00:00,  5.76it/s]


In [20]:
df.to_csv('data/aug_train.csv', index=False)

In [21]:
df.shape

(100480, 2)

: 